# Getting lake level from pit filling elevation

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = FutureWarning)

import heapq
import numpy as np
from math import sqrt

from scipy import ndimage
from scipy import sparse as sp
from scipy.sparse import linalg as splg
from matplotlib import cm
import cmocean as cmo

from scripts import catchmentErosion as eroCatch

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

First, we create a function to plot a elevation grid:
    
+ `plotDEMheatmap`

In [ ]:
def plotDEMheatmap(x,y,dem,title="DEM elevation",color=None,annotate=False):
    
    fig, ax = plt.subplots(figsize=(8,8))
    if color is None:
        im = ax.imshow(dem) 
    else:
        im = ax.imshow(dem,cmap=color)
    
    if annotate:
        for i in range(len(y)):
            for j in range(len(x)):
                text = ax.text(j, i, dem[i, j],
                               ha="center", va="center", color="w")

    ax.set_xticks(np.arange(dem.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(dem.shape[0]+1)-.5, minor=True)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    plt.colorbar(im, cax=cax)

    ax.set_title(title,fontsize=10)
    fig.tight_layout()
    plt.show()

# 1. Output elevation grid

We will use the output from the simulation at any given time step. 

In [ ]:
dataTIN = eroCatch.catchmentErosion(folder='output',timestep=50)
dataTIN.regridTINdataSet()

dem = dataTIN.z.copy()
x = dataTIN.xi[0,:]
y = dataTIN.yi[:,0]
#dataTIN.plotdataSet(title='Elevation', data=dataTIN.z, color=cmo.cm.delta,crange=[-2000,2000])

In [ ]:
plotDEMheatmap(x,y,dem,title="DEM elevation",color=None,annotate=False)

# 2. Pit filling algorithm


<div class="alert alert-block alert-info">Digital elevation models (DEMs) that are used in hydrological applications must be processed to remove sinks, mainly topographic depressions.</div>
 

Flow enforcement techniques include filling methods, which raise elevations within depressions, breaching, which carves channels through blockages, and hybrid methods.

Here we will use the algorithm defined by Planchon & Darboux (2001).

<img src="images/pitPD.jpg" alt="geometry" width="500" height="400"/>

For each given points not belonging to the boundary nodes, we will look at its 8 neighbours:

<img src="images/neighbourhoods.gif" alt="geometry" width="200" height="200"/>


In [ ]:
def pit_fill(data):
    
    dem = np.copy(data)
    elev = np.copy(data)
    
    nrow, ncol = dem.shape
    dem[:,:]=100000.0
    dem[0,:]=elev[0,:]
    dem[-1,:]=elev[-1,:]
    dem[:,0]=elev[:,0]
    dem[:,-1]=elev[:,-1]
    flag = True
    
    # Neighbours along x,y directions 
    direct8y=[1, 1, 1, 0, -1, -1, -1, 0]
    direct8x=[-1, 0, 1, 1, 1, 0, -1, -1]
    
    # Main loop
    while flag==True:
        flag=False
        for i in range(1,nrow-1):
            for j in range(1,ncol-1):
                if dem[i,j]>elev[i,j]:
                    for p in range(0,8):
                        r=i+direct8x[p]
                        c=j+direct8y[p]
                        if elev[i,j] >= dem[r,c]+0.01:
                            dem[i,j]=elev[i,j]
                            flag=True
                        else:
                            if dem[i,j]>dem[r,c]+0.01:
                                dem[i,j]=dem[r,c]+0.01
                                flag=True 
    output_array=dem
    return output_array

We will apply the `pit_fill` function to our inital elevation grid:

In [ ]:
demf = pit_fill(dem)

plotDEMheatmap(x,y,demf,title="Filled DEM elevation",color=None,annotate=False)

Which points have been filled? Corresponding to lake level...

In [ ]:
lake_level = demf-dem

plotDEMheatmap(x,y,lake_level,title="Differences between filled and output DEM",color=cmo.cm.amp)